<h1>Basics of Mobile Robotics project<span class="tocSkip"></span></h1>

**Group 23 : Diana Bejan** (325029)**, Emilie Grandjean** (286734)**, Garance Boesinger** (310447)**, Juan Martín** (376659)

**Table of contents**<a id='toc0_'></a>    
1. [Introduction](#toc1_)    
2. [General Setup](#toc2_)    
2.1. [Project description](#toc2_1_)    
2.2. [Environment Setup  GARANCE](#toc2_2_)    
2.3. [Best Path Calculations KIKE](#toc2_3_)    
2.4. [Motion Control DIANA](#toc2_4_)    
2.5. [Obstacles avoidance EMILIE](#toc2_5_)    
3. [Required Components](#toc3_)    
3.1. [Computer Vision GARANCE](#toc3_1_)    
3.2. [Global Navigation KIKE](#toc3_2_)    
3.3. [Motion Control DIANA](#toc3_3_)    
3.4. [Local Navigation](#toc3_4_)    
3.5. [Filtering](#toc3_5_)    
4. [VIDEOS ? IMAGES ? LITTLE EXTRAS ?](#toc4_)    
5. [Overall Project](#toc5_)    
6. [Conclusion](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[Introduction](#toc0_)

As part of the Basics of Mobile Robotic class given by M. Mondada, we are asked to implement a project using the Thymio robot. This project involves the robot reaching a goal while moving in an environment filled with permanent obstacles. During it's path, some extra physical obstacles can be placed in its way. In such moments, the Thymio has to be able to avoid them and continue its route to the end point.

This is then done using several concepts seen in class such as image processing and pattern detection with Computer Vision, the Motion Control using a controller, Global and Local Navigation of the Thymio and Filtering.

In this report, we will talk about the different modules implemented and detail how they work to explain our overall function of our project.

# 2. <a id='toc2_'></a>[General Setup](#toc0_)

## 2.1. <a id='toc2_1_'></a>[Project description](#toc0_)

1. **Create an environnment :** Our environment has to contain a set of obstacles that the Thymio avoids through *global navigation*. That is to say, the Thymio should avoid the obstacles without using the sensors to detect them.

2. **Find the best path :** The obective is that the Thymio goes from an *arbitrary point* in the map to a *target* that can be placed <u>anywhere</u> in the environment.  These will be changed during the demo to see how the system performs.

3. **Motion Control & Position estimation:** We will *control* the robot to help it move along the path. This requires an accurate estimate of the position of the robot which we will have to obtain through *bayesian filtering*.
   
4. **Avoid Obstacles :** While navigating, the Thymio will have to use *local navigation* to avoid *physical obstacals* that can be put in its path <u>at any point in time</u>. 

## 2.2. <a id='toc2_2_'></a>[Environment Setup  GARANCE](#toc0_)

environment : the static obstacles have to be in 2D (it will be easier to detect with the camera than if it’s in 3D because the shadows are hard to detect and so it’s better to do it that way (heard an assistant say it))

quick explication on what our setup looks like, the use of the colors with pitcures


## 2.3. <a id='toc2_3_'></a>[Best Path Calculations KIKE](#toc0_)

short explication on what method used and why and how and quickly mention that it uses the info send by the CV picked up by the environment

## 2.4. <a id='toc2_4_'></a>[Motion Control DIANA](#toc0_)

moving the Thymio along the given trajectory while knowing the thymio’s position (use of a proportional controller with the angle between the thymio’s orientation and the destination point)

small descritption of the method used and why and what elements from the path needed


## 2.5. <a id='toc2_5_'></a>[Obstacles avoidance EMILIE](#toc0_)

For the obstacle avoidance section, we decided to use 3D physical obstacles that cannot be detected by the camera. This will push the use of the Thymio's sensors to detect these obstacles. Since they can be put at anytime anywhere in the robot's path, using **Local Navigation** will allow to avoid them and once the obstacle avoided, the robot need to go back to following it's given path.

*while the Thymio navigates, use of local navigation to avoid physical (3D) obstacles that can be put in its path at anytime (create hollow obstacles so they are not detected by the camera for the global navigation in the environment) —> use the exercise sessions on local navigation, adapt it to our case*

*explication on what the obstacles look like, short description sans plus*

# 3. <a id='toc3_'></a>[Required Components](#toc0_)

## 3.1. <a id='toc3_1_'></a>[Computer Vision GARANCE](#toc0_)

use of the camera (placed above the environment) —> locate the environment (edges etc.), the static (2D) obstacles and the thymio
	—> only do image processing (quickly saw in an exercise session) and use the given file explaining what to do, how to extract and process the image data.
	—> use AR tags : the camera can practically do everything on it’s own
	—> ! the camera does not use the same parameters for macs and windows => the one doing this will be the only one


details explications : step by step, accompagné by images and pieces of code of

on how it's done, what  calculations used, what variables used, the form of the vision file(is it a class, just a fct ? why ?)

++ add images to go with the description to illustrate the explications

detail explications on the differents functions called and what their inputs are and what their outputs are (can be done in un tableau for example), explain their use and the goal of using them and when they are used (like it what function are they called etc )


separation in several parts ? the computation of the position of thymio? the environment ? the obstacles ? the goal ?

++ sources


## 3.2. <a id='toc3_2_'></a>[Global Navigation KIKE](#toc0_)

use the location of the static obstacles given by the vision step and find the optimal path the thymio has to follow to reach the goal (no movement, just find the best trajectory)
	—> use of existing libraries !

  
details explications : step by step, accompagné by images(?)--> video showing the path followed ? and pieces of code of

on how it's done, what  calculations used, what variables used, the form of the vision file(is it a class, just a fct ? why ?)

++ add images or videos to go with the description to illustrate the explications

detail explications on the differents functions called and what their inputs are and what their outputs are (can be done in un tableau for example), explain their use and the goal of using them and when they are used (like it what function are they called etc )


separation in several parts ? the computation of the position of thymio? the environment ? the obstacles ? the goal ? how was the algo written and used ? what does it send to who (MC) 
--> the implementation, the algorithm, ...

is it optimal ? yes ok, no, why ? 

++ sources


## 3.3. <a id='toc3_3_'></a>[Motion Control DIANA](#toc0_)

use the trajectory given in the global navigation step to direct and move the thymio to the goal destination
	—> use of a P controller

details explications : step by step, accompagné by images(?)--> video showing the path followed ? and pieces of code of

on how it's done, what  calculations used, what variables used, the form of the vision file(is it a class, just a fct ? why ?)

++ add images or videos to go with the description to illustrate the explications

detail explications on the differents functions called and what their inputs are and what their outputs are (can be done in un tableau for example), explain their use and the goal of using them and when they are used (like it what function are they called etc )


separation in several parts ? the computation of the position of thymio? the environment ? the obstacles ? the goal ? how was the algo written and used ? what does it send to who (MC) 
--> the implementation, the algorithm, ...

is it optimal ? yes ok, no, why ? 

++ sources


## 3.4. <a id='toc3_4_'></a>[Local Navigation](#toc0_)

**Idea**

The **Local Navigation** module is inspired by the exercise session *Week 3 - Artificial Neural Networks* paragraph 5. The local_navigation.py file contains 3 functions, two verification functions and the actual function that makes the Thymio avoid the obstacle.

**1. Testing the presence of an obstacle**

| Funtion | Input | Output |
|:------|:------|:------|
|   ` def test_saw_osb(Thymio, node, obs_threshold): `  | The class *Thymio* from the file *classes.py* to be able to modify the booleen variable **obs_avoided**, the threashold value to which the sensors detect the obstacle is close enough | returns *True* or *False* depending on if there's an obstacle or not, set the booleen variable **obs_avoided** to *False* if an obstacle is detected
  

The function **test_saw_obs** is actually practically the same as *test_saw_wall* from the exercise but without the use of the *verbose* booleen and instead, when an obstacle is detected, the function sets a global booleen variable called **obs_avoided** to *false*. This variable is used in the Local obstacle avoidance function so that it stays in the loop until the obstacle is fully contoured.

This function take in input the obstacle threashold (*obs_threshold*) that is set in the 3rd function.

In [7]:
def test_saw_osb(Thymio, node, obs_threshold) :

    '''This function verrifies if one of the proximity horiontal sensors
        sees and obstacle within the giving threashold.'''
    
    if any([x>obs_threshold for x in node['prox.horizontal'][:-2]]):

        Thymio.obs_avoided = False  # Booleen to state when the obstacle has been avoided or not
        return True

    return False

**2. Testing in what direction to contourn**

| Function | Input | Output |
|:------|:------|:------|
|   `def clockwise(node):`  |  | returns *True* or *False* depending on if the obstacle is most on the right or the left of the robot and let the next function know weather to contourn clockwise or counterclockwise
  

Our obstacles are cylinders that the Thymio contourns. Therefore, we want to know whether to contourn it clockwise or counterclockwise. This depends on if the obstacle is more on the right or the left of the robot when it get's in its path. The function **clockwise** tests which sensor between the sensor [1] (left) and the sensor [3] (right) is currently detecting the obstacle. If in fact the sensor on the left ([1]) has a higher value than the other one, the functionn returns *true*, otherwise it returns *false*. This function is only called in the Local obstacle avoidance function.

In [8]:
def clockwise(node) :

    '''This function verrifies if the obstacle to avoid is more on its right or left,
        therefore, the Thymio will contourn it accordingly.'''

    prox = list(node["prox.horizontal"]) + [0]
    print(prox[1])
    print(prox[3])
    if prox[1] > prox[3] :
        return True
    
    return False

**3. Local obstacle avoidance**

| Function| Input | Output |
|:------|:------|:------|
|   `def obstacle_avoidance(Thymio, node, client, motor_speed=100, obs_threshold=500):`  | The class *Thymio* from the file *classes.py* to be able to modify the booleen variable **obs_avoided**, the motor speed for the movement of the wheels, set to 100, the threashold value to which the sensors detect the obstacle set to 500 | no real output, but modification of **obs_avoided** booleen when the obstacle is avoided
  

The function **obstacle_avoidance** can be devided in two states : the '*turning*' and the '*contourning*'.

As soon the function is called, it starts by setting two variables : **clockwise_true** to *false* and **prev_state** to "*turning*". Then, since the 1st function (**test_saw_osb**) has been called, the booleen **obs_avoided** is *false* and so we enter a while loop. 

This is when we enter the state of '*turning*' where we test, thanks to the 2nd function (**clockwise**) if the Thymio will have to contourn on the left or on the right of the obstacle. If the robot has to turn clockwise, the booleen **clockwise_true** is set to *true*, otherwise, it stays to *false*. The set '*turning*' makes the Thymio rotate on itself so its not facing the obstacle anymore. Once this is done, the state is set to '*contourning*'.

Still in the while loop, we enter the state of '*contourning*' where, depending on the value of te booleen **clockwise_true**, the Thymio will move in an arc circle around the obstacle and then back to the '*turning*' state, it rotates on itself again to be back facing in the right direction to allow going on with the Motion Control.

At the end of the '*contourning*' state, the booleen variable **obs_avoided** is set to *true*, meaning that the Thymio has finised avoiding the obstacle and that the Motion Control can pick up again.



In [9]:
def obstacle_avoidance(Thymio, node, client, motor_speed=100, obs_threshold=500): #, clockwise = False):
    """
    Wall following behaviour of the FSM
    param motor_speed: the Thymio's motor speed
    param wall_threshold: threshold starting which it is considered that the sensor saw a wall
    param white_threshold: threshold starting which it is considered that the ground sensor saw white
    param verbose: whether to print status messages or not
    """

    clockwise_true = False  # Booleen to state if the Thymio has to contourn on the left or right
 
    prev_state = "turning" # Stated of movement of the Thymio
    
    while not Thymio.obs_avoided :     # As long as the obstacle isn't avoided, stay in the while loop 
    
        if test_saw_osb(Thymio, node, obs_threshold) :
            
            if prev_state == "turning": # little rotation on it's own to then do the contourning
  
                if clockwise(node) :
                    
                    Thymio.setSpeedLeft(motor_speed, node)
                    Thymio.setSpeedRight(-motor_speed, node)

                    clockwise_true = True   # Thymio needs to contourn the obstacle clockwise

                else :
                    # Thymio needs to contourn the obstacle counterclockwise
                    
                    Thymio.setSpeedLeft(-motor_speed, node)
                    Thymio.setSpeedRight(motor_speed, node)
                    
                prev_state = "contourning" # Change the state so the Thymio countourns the obstacle fully
        
        else:
            if prev_state == "contourning": 

                if clockwise_true :

                    Thymio.setSpeedLeft(motor_speed-40, node)
                    Thymio.setSpeedRight(motor_speed, node)

                    prev_state = "turning"

                    aw(client.sleep(18))

                    Thymio.setSpeedLeft(motor_speed, node)
                    Thymio.setSpeedRight(-motor_speed, node)

                    aw(client.sleep(2))

                    Thymio.obs_avoided = True  # obstacle has been avoided, change the state booleen
                    
                else :

                    Thymio.setSpeedLeft(motor_speed,node)
                    Thymio.setSpeedRight(motor_speed-40,node)

                    prev_state="turning"

                    aw(client.sleep(18))

                    Thymio.setSpeedLeft(-motor_speed,node)
                    Thymio.setSpeedRight(motor_speed,node)

                    aw(client.sleep(2))

                    Thymio.obs_avoided = True  # obstacle has been avoided, change the state booleen

        aw(client.sleep(0.1)) #otherwise, variables would not be updated

## 3.5. <a id='toc3_5_'></a>[Filtering](#toc0_)

**Idea**

bla bla bla



# 4. <a id='toc4_'></a>[VIDEOS ? IMAGES ? LITTLE EXTRAS ?](#toc0_)

# 5. <a id='toc5_'></a>[Overall Project](#toc0_)

**do we have to put the main here ? or a code that called our main.py ? to be asked on tuesday maybe**

# 6. <a id='toc6_'></a>[Conclusion](#toc0_)

**we need to write a conclusion**
